<a href="https://colab.research.google.com/github/amberreynolds/QuantiFi-Radio/blob/master/QuantiFyRadio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (122 kB/s)
Reading package lists... Done


In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-14 01:14:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-08-14 01:14:22 (1.05 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dj-ai-database").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url1 = "https://quantifi-radio.s3.us-east-2.amazonaws.com/Hot+Stuff+-+cleaned.csv"
spark.sparkContext.addFile(url1)

hotstuffdf = spark.read.option('header', 'true').csv(SparkFiles.get("Hot+Stuff+-+cleaned.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
hotstuffdf.show(10)

+--------------------+--------------------+--------------------+----+---------------------+
|              SongID|                Song|           Performer|Year|Occurences on Hot 100|
+--------------------+--------------------+--------------------+----+---------------------+
|"""B"" GirlsYoung...|       """B"" Girls"|  Young And Restless|1990|                   15|
|"""Cherry Cherry"...|"""Cherry Cherry"...|        Neil Diamond|1973|                   10|
|"""Having A Party...|"""Having A Party...|The Ovations (Fea...|1973|                    9|
|"""Joy"" Pt. IIsa...|     """Joy"" Pt. I"|         Isaac Hayes|1973|                    9|
|"""Roots"" Medley...|  """Roots"" Medley"|        Quincy Jones|1977|                    7|
|"""Yep!""Duane Ed...|          """Yep!"""|Duane Eddy His Tw...|1959|                    9|
|#1 Dee JayGoody G...|          #1 Dee Jay|         Goody Goody|1978|                    5|
|             #1Nelly|                  #1|               Nelly|2001|           

In [ ]:
# Rename columns with spaces
hotstuffdf = hotstuffdf.withColumnRenamed("Occurences on Hot 100", "occurences_on_hot_100")
hotstuffdf.show(5)

+--------------------+--------------------+--------------------+----+---------------------+
|              SongID|                Song|           Performer|Year|occurences_on_hot_100|
+--------------------+--------------------+--------------------+----+---------------------+
|"""B"" GirlsYoung...|       """B"" Girls"|  Young And Restless|1990|                   15|
|"""Cherry Cherry"...|"""Cherry Cherry"...|        Neil Diamond|1973|                   10|
|"""Having A Party...|"""Having A Party...|The Ovations (Fea...|1973|                    9|
|"""Joy"" Pt. IIsa...|     """Joy"" Pt. I"|         Isaac Hayes|1973|                    9|
|"""Roots"" Medley...|  """Roots"" Medley"|        Quincy Jones|1977|                    7|
+--------------------+--------------------+--------------------+----+---------------------+
only showing top 5 rows



In [ ]:
from pyspark import SparkFiles
url2 = "https://quantifi-radio.s3.us-east-2.amazonaws.com/Hot+100+Audio+Features+-+cleaned.csv"
spark.sparkContext.addFile(url2)

audiodf = spark.read.option('header', 'true').csv(SparkFiles.get("Hot+100+Audio+Features+-+cleaned.csv"), inferSchema=True, sep=',')
audiodf.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+--------------------+----------------------+-------------------------+------------------------+------------+------+----+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|              SongID|           Performer|                Song|       spotify_genre|    spotify_track_id|spotify_track_preview_url| spotify_track_album|spotify_track_explicit|spotify_track_duration_ms|spotify_track_popularity|danceability|energy| key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+--------------------+----------------------+-------------------------+------------------------+------------+------+----+--------+----+-----------+------------+-----------

In [ ]:
# audiodf.printSchema()

root
 |-- SongID: string (nullable = true)
 |-- Performer: string (nullable = true)
 |-- Song: string (nullable = true)
 |-- spotify_genre: string (nullable = true)
 |-- spotify_track_id: string (nullable = true)
 |-- spotify_track_preview_url: string (nullable = true)
 |-- spotify_track_album: string (nullable = true)
 |-- spotify_track_explicit: string (nullable = true)
 |-- spotify_track_duration_ms: string (nullable = true)
 |-- spotify_track_popularity: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)



In [ ]:
from pyspark.sql.functions import col
hotstuffdf = hotstuffdf.withColumn('Year', col('Year').cast('float'))

hotstuffdf = hotstuffdf.withColumn('occurences_on_hot_100', col('occurences_on_hot_100').cast('float'))

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-dj-ai.c0b2m66eukcx.us-east-2.rds.amazonaws.com:5432/database_dj_ai"
config = {"user":"root",
          "password": "freeze123",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

hotstuffdf.write.jdbc(url=jdbc_url, table='hotstuff', mode=mode, properties=config)

In [ ]:
audiodf.write.jdbc(url=jdbc_url, table='audio_features', mode=mode, properties=config)